<a href="https://colab.research.google.com/github/liizaaa/machine_learning/blob/main/%D1%82%D1%80%D0%B5%D1%82%D1%8C%D1%8F_%D0%BB%D0%B0%D0%B1%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df = pd.read_csv("household_income.csv")
df.head()

,Age,Education_Level,Occupation,Number_of_Dependents,Location,Work_Experience,Marital_Status,Employment_Status,Household_Size,Homeownership_Status,Type_of_Housing,Gender,Primary_Mode_of_Transportation,Income
0,56,Master's,Technology,5,Urban,21,Married,Full-time,7,Own,Apartment,Male,Public transit,72510
1,69,High School,Finance,0,Urban,4,Single,Full-time,7,Own,Apartment,Male,Biking,75462
2,46,Bachelor's,Technology,1,Urban,1,Single,Full-time,7,Own,Single-family home,Female,Car,71748
3,32,High School,Others,2,Urban,32,Married,Full-time,1,Own,Apartment,Female,Car,74520
4,60,Bachelor's,Finance,3,Urban,15,Married,Self-employed,4,Own,Townhouse,Male,Walking,640210


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Age                             10000 non-null  int64 
 1   Education_Level                 10000 non-null  object
 2   Occupation                      10000 non-null  object
 3   Number_of_Dependents            10000 non-null  int64 
 4   Location                        10000 non-null  object
 5   Work_Experience                 10000 non-null  int64 
 6   Marital_Status                  10000 non-null  object
 7   Employment_Status               10000 non-null  object
 8   Household_Size                  10000 non-null  int64 
 9   Homeownership_Status            10000 non-null  object
 10  Type_of_Housing                 10000 non-null  object
 11  Gender                          10000 non-null  object
 12  Primary_Mode_of_Transportation  10000 non-null 

In [20]:
df.isnull().sum()

,0
Age,0
Education_Level,0
Occupation,0
Number_of_Dependents,0
Location,0
Work_Experience,0
Marital_Status,0
Employment_Status,0
Household_Size,0
Homeownership_Status,0


In [24]:
X = df.drop('Income', axis=1)
y = df['Income']

разделяем на обучающую и тестовую выборки

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

определение категориальных и числовых признаков

In [33]:
categorical_features = ['Education_Level', 'Occupation', 'Location', 'Marital_Status',
                        'Employment_Status', 'Homeownership_Status', 'Type_of_Housing', 'Gender',
                        'Primary_Mode_of_Transportation']
numeric_features = ['Age', 'Number_of_Dependents', 'Work_Experience', 'Household_Size']

преобразование категориальных (образование профессия местоположение и тд) и числовых (возраст количество опыт и тд) признаков

In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

построение четырех моделек с использованием различных регрессоров

In [52]:
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}
best_model = None
best_score = float('-inf')

настраиваем гиперпараметры для каждого алгоритма + GridSearchCV

In [53]:
param_grids = {
    'Linear Regression': {},
    'Decision Tree': {
        'regressor__max_depth': [5, 10, 15, 20],
        'regressor__min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [5, 10, 15],
        'regressor__min_samples_split': [2, 5, 10]
    },
    'Gradient Boosting': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__learning_rate': [0.01, 0.05, 0.1],
        'regressor__max_depth': [3, 5, 7]
    }
}

запускаем и ждем нашу наилучшую модельку из четырех

In [54]:
for model_name, model in models.items():
    print(f"Training {model_name}...")
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    grid_search = GridSearchCV(pipeline, param_grid=param_grids[model_name], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Получаем наилучшую модель и ее метрики
    best_model_candidate = grid_search.best_estimator_
    y_pred = best_model_candidate.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"R²: {r2}")

Training Linear Regression...
Best parameters for Linear Regression: {}
Mean Squared Error (MSE): 3136367852509.854
R²: 0.005862187302471122
Training Decision Tree...
Best parameters for Decision Tree: {'regressor__max_depth': 5, 'regressor__min_samples_split': 10}
Mean Squared Error (MSE): 3125798843847.597
R²: 0.009212257079990738
Training Random Forest...
Best parameters for Random Forest: {'regressor__max_depth': 10, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Mean Squared Error (MSE): 3046010400599.654
R²: 0.03450288374725974
Training Gradient Boosting...
Best parameters for Gradient Boosting: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 7, 'regressor__n_estimators': 100}
Mean Squared Error (MSE): 3058755786705.36
R²: 0.030462965325390834


сравнение модели с наилучшей производительностью

In [55]:
    if r2 > best_score:
        best_score = r2
        best_model = best_model_candidate

In [56]:
print(f"\nBest model is: {best_model}")


Best model is: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age',
                                                   'Number_of_Dependents',
                                                   'Work_Experience',
                                                   'Household_Size']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Education_Level',
                                                   'Occupation', 'Location',
                                                   'Marital_Status',
                                                   'Employment_Status',
                                                   'Homeownership_Status',
                                                   'Type_of_Housing', 'Gender',
                                                   'Primary_Mode_of_Transportation'])

ииии так барабанная дробь.....!!!!!!!
.
.
.
наилучшей моделью для прогнозирования доходов домохозяйств является Gradient Boosting Regressor с параметрами:

learning_rate: 0.01 - низкая скорость обучения, что позволяет модели медленно, но точно настраиваться на данные

max_depth: 7 - максимальная глубина деревьев в ансамбле, что помогает избежать переобучения при работе с данными

random_state: 42

##давайте испортим мне настроение
а почему бы не попытаться улучшить нашу и без того замечательную модель?
предлагаю пошалить с гиперпараметрами и посмотреть, что же из этого получится

In [57]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

вот такая ⭐сеточка⭐ у меня получилась

In [58]:
param_grid = {
    'regressor__n_estimators': [100, 200, 300],  #число деревьев
    'regressor__max_depth': [3, 5, 7, 10],  #глубина деревьев
    'regressor__learning_rate': [0.01, 0.05, 0.1],  #скорость обучения
    'regressor__min_samples_split': [2, 5, 10],  #минимум образцов для разбиения узла
    'regressor__min_samples_leaf': [1, 2, 4]  #минимум образцов для листа
}

и конечно же добавим ⚡кросс-валидацию⚡

In [59]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

обучаем модельку

In [60]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Number_of_Dependents',
                                                                          'Work_Experience',
                                                                          'Household_Size']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['Education_Level',
                                                                          'Occupation',
                                                                          'Location',
                                                                          'Marital_Status',
                                                                          'Employment_Status',
                                                                          'Homeownership_Status',
                                                                          'Type_of_Housing',
                                                                          'Gender',
                                                                          'Primary_Mode_of_Transportation'])])),
                                       ('regressor',
                                        GradientBoostingRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__learning_rate': [0.01, 0.05, 0.1],
                         'regressor__max_depth': [3, 5, 7, 10],
                         'regressor__min_samples_leaf': [1, 2, 4],
                         'regressor__min_samples_split': [2, 5, 10],
                         'regressor__n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [61]:
best_model = grid_search.best_estimator_

проводим оценку модели на тестовых данных

In [62]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [63]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R²: {r2}")

Best parameters: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 7, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100}
Mean Squared Error (MSE): 3058755786705.36
R²: 0.030462965325390834


Вы тоже это видите???
Спустя всего лишь ~1 час 20 минут мы увидели такой же результат, который был у нас изначально:)

если честно, то у меня закололо сердце после того, что я увидела.
мы нашли действительно наилучшую модель!